In [ ]:
## import numpy as np

def custom_svd(matrix):
    """
    Performs a custom Singular Value Decomposition (SVD) on a given matrix and computes its inverse, 
    condition number, and reconstructs the original matrix.

    Parameters:
    ----------
    matrix : np.ndarray
        A 2D NumPy array representing the matrix to decompose.

    Returns:
    -------
    U : np.ndarray
        Left singular vectors (U matrix) of the input matrix.
    S_diag : np.ndarray
        Diagonal matrix of singular values.
    Vt : np.ndarray
        Right singular vectors (V transpose) of the input matrix.
    condition_number : float
        Condition number of the matrix, calculated as the ratio of the largest to smallest singular values.
    matrix_inverse : np.ndarray
        Pseudo-inverse of the input matrix.

    Raises:
    ------
    ValueError:
        If the matrix is singular and does not have an inverse.
    """

    # Step 1: Compute A^T * A, which is used to determine the right singular vectors
    AtA = np.dot(matrix.T, matrix)
    
    # Step 2: Eigen decomposition for right singular vectors (V) and corresponding singular values
    eigvals_AtA, V = np.linalg.eig(AtA)
    
    # Step 3: Sort eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigvals_AtA)[::-1]
    eigvals_AtA = eigvals_AtA[sorted_indices]
    V = V[:, sorted_indices]
    
    # Step 4: Calculate singular values (square root of eigenvalues of A^T * A)
    singular_values = np.sqrt(eigvals_AtA)
    if singular_values[-1] == 0:
        raise ValueError("Matrix is singular and does not have an inverse.")
    
    # Step 5: Calculate U (left singular vectors) using A * V / singular_values
    U = matrix @ V / singular_values
    U /= np.linalg.norm(U, axis=0)  # Normalize U

    # Step 6: Construct the diagonal matrix S with singular values
    S_diag = np.diag(singular_values)
    
    # Step 7: Calculate the pseudo-inverse of S (S_inv) for non-zero singular values
    S_inv_diag = np.diag(1 / singular_values)
    
    # Step 8: Calculate the pseudo-inverse of the matrix
    matrix_inverse = V @ S_inv_diag @ U.T

    # Step 9: Calculate the condition number as the ratio of the largest to the smallest singular value
    condition_number = singular_values[0] / singular_values[-1]

    return U, S_diag, V.T, condition_number, matrix_inverse


def create_matrix_a(shape):
    """Create matrix A based on the specified shape, representing a system of springs and masses."""
    matrix_a = np.zeros(shape)
    np.fill_diagonal(matrix_a, 1)
    if shape[0] > 1:
        np.fill_diagonal(matrix_a[1:], -1)
    return matrix_a

def create_matrix_c(spring_constants):
    """Create a diagonal matrix C based on a list of spring constants."""
    return np.diag(spring_constants)

def calculate_force_vector(masses):
    """Calculate the force vector based on a list of masses."""
    return np.array(masses) * -9.81

def compute_stiffness_matrix(matrix_a, matrix_c):
    """Compute the stiffness matrix K based on matrices A and C."""
    return matrix_a.T @ matrix_c @ matrix_a

def solve_displacement(stiffness_matrix, force_vector):
    """Solve for displacements."""
    U, S_diag, V_T, condition_number, inv_stiffness = custom_svd(stiffness_matrix)
    return inv_stiffness @ force_vector

def calculate_spring_elongation(matrix_a, displacement):
    """Calculate elongations of the springs."""
    return matrix_a @ displacement

def compute_internal_forces(matrix_c, elongation):
    """Calculate internal forces in the springs."""
    return matrix_c @ elongation

def gather_spring_and_mass_data():
    """Prompt the user for input regarding the number of springs, masses, spring constants, and masses."""
    while True:
        try:
            num_springs = int(input("Enter the number of springs:\n"))
            num_masses = int(input("Enter the number of masses:\n"))
            boundary_condition_input = input("Enter boundary condition (1 for Fixed-Free, 2 for Fixed-Fixed):\n")
            if boundary_condition_input == "1":
                boundary_condition = "Fixed-Free"
            elif boundary_condition_input == "2":
                boundary_condition = "Fixed-Fixed"
            else:
                print("Invalid input. Enter 1 for Fixed-Free or 2 for Fixed-Fixed.")
                continue

            if num_springs > 0 and num_masses > 0 and (
                (num_springs == num_masses and boundary_condition == "Fixed-Free") or
                (num_springs == num_masses + 1 and boundary_condition == "Fixed-Fixed")
            ):
                break
            else:
                print("Input not accepted. Please follow the instructions.")
                print("The number of masses must be equal to the number of springs for 'Fixed-Free' or one less for 'Fixed-Fixed'.")
                print("The boundary condition must be either 'Fixed-Free' or 'Fixed-Fixed'.")
        except ValueError as e:
            print(f'Error: {e}')
            print("Please enter positive integers for the number of springs and masses.")
            print("The boundary condition must be either 'Fixed-Free' or 'Fixed-Fixed'.")
    
    spring_constants = []
    mass_values = []
    for i in range(num_springs):
        while True:
            try:
                spring_constant = float(input(f"Enter the spring constant for spring {i+1}:\n"))
                if spring_constant > 0:
                    spring_constants.append(spring_constant)
                    break
                else:
                    print("Please enter a positive value for the spring constant.")
            except ValueError as e:
                print(f"Error: {e}\nPlease enter a valid positive number for the spring constant.")

    for i in range(num_masses):
        while True:
            try:
                mass = float(input(f"Enter the mass for mass {i+1}:\n"))
                if mass > 0:
                    mass_values.append(mass)
                    break
                else:
                    print("Please enter a positive value for the mass.")
            except ValueError as e:
                print(f"Error: {e}\nPlease enter a valid positive number for the mass.")

    return num_springs, num_masses, spring_constants, mass_values

def main():
    num_springs, num_masses, spring_constants, mass_values = gather_spring_and_mass_data()
    matrix_shape = (num_springs, num_masses)

    # Define matrices and vectors
    matrix_a = create_matrix_a(matrix_shape)
    matrix_c = create_matrix_c(spring_constants)
    force_vector = calculate_force_vector(mass_values)
    stiffness_matrix = compute_stiffness_matrix(matrix_a, matrix_c)

    # Solve for displacements
    displacement_vector = solve_displacement(stiffness_matrix, force_vector)

    # Calculate elongations and internal forces
    elongation_vector = calculate_spring_elongation(matrix_a, displacement_vector)
    internal_forces = compute_internal_forces(matrix_c, elongation_vector)

    # Perform SVD on the stiffness matrix
    U, Sigma, V_T, condition_number, inv_stiffness_matrix = custom_svd(stiffness_matrix)

    # Print results
    print(f'\nMatrix A:\n{matrix_a}')
    print(f'\nSpring constant matrix C:\n{matrix_c}')
    print(f'\nSingular value matrix Sigma:\n{Sigma}')
    print(f'\nL2 Condition number of the stiffness matrix : {condition_number}')
    print(f'\nForce vector f:\n{force_vector}')
    print(f'\nStiffness matrix K:\n{stiffness_matrix}')
    print(f'\nDisplacement vector:\n{displacement_vector}')
    print(f'\nElongation vector:\n{elongation_vector}')
    print(f'\nInternal forces vector:\n{internal_forces}')


if __name__ == "__main__":
    main()
